In [ ]:
%pip install llama_index
%pip install transformers
%pip install FlagEmbedding
%pip install llama_index.llms.huggingface

### USING LLMs 
customizing LLMS

###Embeddings

In [ ]:
%pip install llama_index.embeddings.huggingface

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings


Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)  # if embed_model is not specified here, llama index uses openai as default embed model
# Hence, the Settings is modifying the "environment" variable to the llama index workflow

Settings.llm = None
Settings.chunk_overlap = 64 # allow chunk overlay to avoid cutting off key info in between
Settings.chunk_size = 512 # each chunk contains 256 characters

documents = SimpleDirectoryReader('./Primetime').load_data()
index = VectorStoreIndex.from_documents(documents)

# setting up retriever
top_k = 3
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.6)]
)

query = "how to set an input delay to an input port named IN1"
response = query_engine.query(query)

context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

with open('./response.txt', 'w') as output:
    output.write(context)

LLM is explicitly disabled. Using MockLLM.
